In [2]:
import torch
import carp_models
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import h5py
import numpy as np
from tqdm import tqdm
device = torch.device("cuda")

In [3]:
config={'model':'ByteNetLM',
                'lr':1e-3,
                'n_tokens':len(RNA_ALPHABET),
                'd_embedding' : 9, # dimension of embedding
                'd_model': 320, # dimension to use within ByteNet model, //2 every layer
                'n_layers' : 15, # number of layers of ByteNet block
                'activation': 'relu',
                'kernel_size' : 5, # the kernel width
                'r' : 32, # used to calculate dilation factor
                'padding_idx' : RNA_ALPHABET.index('-') ,# location of padding token in ordered alphabet
                'dropout' : 0.1 ,
                }

model = carp_models.ByteNetLM(config['n_tokens'], config['d_embedding'], config['d_model'],
                        config['n_layers'], config['kernel_size'], config['r'], config['lr'],
                        padding_idx=config['padding_idx'], causal=False, dropout=config['dropout'])
model.load_state_dict(torch.load('./rna-selftrain/2hkapjgg/checkpoints/best_model.ckpt')['state_dict'])
model.to(device)
model.eval()

ByteNetLM(
  (embedder): ByteNet(
    (embedder): Embedding(9, 9, padding_idx=6)
    (up_embedder): PositionFeedForward(
      (conv): Conv1d(9, 320, kernel_size=(1,), stride=(1,))
    )
    (layers): ModuleList(
      (0): ByteNetBlock(
        (conv): MaskedConv1d(160, 160, kernel_size=(5,), stride=(1,), padding=(2,))
        (sequence1): Sequential(
          (0): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
          (1): ReLU()
          (2): PositionFeedForward(
            (conv): Conv1d(320, 160, kernel_size=(1,), stride=(1,))
          )
          (3): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
          (4): ReLU()
        )
        (sequence2): Sequential(
          (0): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
          (1): ReLU()
          (2): PositionFeedForward(
            (conv): Conv1d(160, 320, kernel_size=(1,), stride=(1,))
          )
        )
      )
      (1): ByteNetBlock(
        (conv): MaskedConv1d(160, 160, kernel_size=(5,

In [4]:
save_output = SaveOutput()
hook_handles = []
handle = model.last_norm.register_forward_hook(save_output)

In [5]:
file = h5py.File('../data/rna_stable/insert_dataset.h5','r')
carp_output = h5py.Fileh5py.File('../../data/rna_stable/roberta_embed.h5','w')
batch_size = 64
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)
    expand_seq = np.expand_dims(np.array(string_seq),axis = -1)
    token_output = collater(expand_seq)[0]
    for seq_i in tqdm(range(0,len(token_output),batch_size)):
        seq_batch = torch.tensor(token_output[seq_i:seq_i+batch_size]).to(device)
        output_seq = model(seq_batch).cpu().detach().numpy()
    roberta_output.create_dataset(name=key,data = np.array(output_cache))
    roberta_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

  0%|          | 0/18 [00:00<?, ?it/s]/tmp/ipykernel_3990607/2117338437.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_batch = torch.tensor(token_output[seq_i:seq_i+batch_size]).to(device)
100%|██████████| 18/18 [00:00<00:00, 27.17it/s]


In [10]:
save_output.outputs[0].shape

(64, 173, 320)